In [ ]:
# Preliminary inputs
pretrained_FDD_path = None # Insert the path to the pretrained FullDenoisingDiffusion model
wandb_key = None # Insert your wandb code here

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!git clone https://github.com/KaziFahmi/EDM_Custom

In [ ]:
import sys
sys.path.append('/content/EDM_Custom')

In [ ]:
!pip install hydra-core torch_geometric wandb pytorch_lightning omegaconf torchmetrics setuptools scikit_learn imageio matplotlib numpy pandas Pillow tqdm rdkit

In [ ]:
%cd EDM_Custom/

In [ ]:
!pip install -r /content/EDM_Custom/requirements.txt

In [ ]:
!pip install rdkit

In [1]:
import sys
import torch
import pytorch_lightning as pl

import os
import warnings
import pathlib
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.utilities.warnings import PossibleUserWarning

from midi.datasets import qm9_dataset, geom_dataset
from midi.diffusion_model import FullDenoisingDiffusion
from controlnet.conditioned_diffusion_model import ConditionedDiffusion
from omegaconf import OmegaConf
from hydra import compose, initialize

In [2]:
config_path = "configs/"  
def load_hydra_configs(config_path):
    with initialize(config_path=config_path):
        cfg = compose(config_name="config")
    return cfg

In [3]:
cfg = load_hydra_configs(config_path)

C:\Users\xcdtl\AppData\Local\Temp\ipykernel_16228\1311385683.py:3: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path=config_path):


In [ ]:
dataset_config = cfg.dataset
data_module = qm9_dataset.QM9DataModule(cfg)
dataset_info = qm9_dataset.QM9infos(datamodule=data_module,cfg=cfg)

In [ ]:
pretrained_weights = torch.load(pretrained_FDD_path)

In [ ]:
pretrained_FDD = FullDenoisingDiffusion(cfg=cfg, dataset_infos=dataset_info, train_smiles=[])

Marginal distribution of the classes: nodes: tensor([0.7231, 0.1151, 0.1591, 0.0026]) -- edges: tensor([0.7261, 0.2383, 0.0274, 0.0082, 0.0000]) -- charges: tensor([0.0078, 0.9705, 0.0218])


In [ ]:
pretrained_FDD.load_state_dict(pretrained_weights['state_dict'])
pretrained_EDM = pretrained_FDD.model

In [ ]:
model = ConditionedDiffusion(cft=cfg, dataset_infos=dataset_info, train_smiles=[], pretrained_EDM=pretrained_EDM)

In [ ]:
params_to_ignore = ['module.model.train_smiles', 'module.model.dataset_infos']

In [ ]:
torch.nn.parallel.DistributedDataParallel._set_params_and_buffers_to_ignore_for_model(model, params_to_ignore)

In [ ]:
checkpoint_callback = ModelCheckpoint(dirpath=f"checkpoints/{cfg.general.name}",
                                              filename='{epoch}',
                                              monitor='val/epoch_NLL',
                                              save_top_k=5,
                                              mode='min',
                                              every_n_epochs=1)

In [ ]:
callbacks=[]

In [ ]:
last_ckpt_save = ModelCheckpoint(dirpath=f"checkpoints/{cfg.general.name}", filename='last', every_n_epochs=1)
callbacks.append(checkpoint_callback)
callbacks.append(last_ckpt_save)

In [ ]:
import wandb
wandb.login(key=wandb_key)

False

In [ ]:
trainer = Trainer(gradient_clip_val=cfg.train.clip_grad, # Needed to load old checkpoints
                      accelerator='gpu' if torch.cuda.is_available() else 'cpu',
                      devices=cfg.general.gpus,
                      max_epochs=cfg.train.n_epochs,
                      check_val_every_n_epoch=cfg.general.check_val_every_n_epochs,
                      fast_dev_run=cfg.general.name == 'debug',
                      enable_progress_bar=cfg.train.progress_bar,
                      callbacks=callbacks,
                      log_every_n_steps=50,
                      )

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\xcdtl\anaconda3\envs\midi\lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(


In [ ]:
# FILEPATH: /f:/FYDP/EDM_Custom/colab_notebook.ipynb
trainer.fit(model, datamodule=data_module)


In [7]:
import tracemalloc

def example_function():
    tracemalloc.start()
    # your code to profile
    a = [1] * (10 ** 6)  # Example line to profile
    snapshot = tracemalloc.take_snapshot()
    top_stats = snapshot.statistics('lineno')

    print("[ Top 10 ]")
    for stat in top_stats[:10]:
        print(stat)